# Cleaning DoD Comptroller Budget Data

#### This pulls the main 2019 excel budget spreadsheets from the comptroller website, cleans, and concatenates them to get the 2017 data
#### This provides some of the picture around agency budgets, but there are particular limits when looking at non-Service budgets

### Loading Packages

In [45]:
import pandas as pd
import os
import numpy as np
import requests

#this will put the files in whatever folder you're in

### Pulling Files From Comptroller Site

In [2]:
def pullFile(year, cleanLink):
    r = requests.get(cleanLink, allow_redirects=True)
    name=str(year)+cleanLink.split("/")[-1]
    open(name, 'wb').write(r.content)

def pullBudgetFiles(year):
    filesWeWant=["c1", "m1", "p1", "r1", "rf1", "o1"]  
    for file in filesWeWant:
        cleanLink=f'https://comptroller.defense.gov/Portals/45/Documents/defbudget/fy{year}/{file}.xlsx'
        pullFile(year, cleanLink)

def pullDataRange(firstYear, lastYear):
    for year in range(firstYear, lastYear+1):
        pullBudgetFiles(year)

os.chdir(r'H:\_MyComputer\Documents\Git Repos\budgetData\concatenatingBudgetData')
#os.chdir(r'C:\Users\admin\Documents\Python Scripts\budgetFiles\budgetFiles\git budget\concatenatingBudgetData')
pullDataRange(2013, 2021)

### Reading Data Files

In [46]:
def cleanColumnName(columnName):
    noExits=columnName.replace('\n'," ")
    lessWhiteSpace=' '.join(' '.join(noExits.split()).split())
    return(lessWhiteSpace)

def aggData(budgetFiles, year):
    allBudgetData=pd.DataFrame()
    for spreadsheet in budgetFiles:
        filename = os.path.join(os.getcwd(), spreadsheet)
        df = pd.read_excel(filename, None, skiprows=1)
    #print(df.keys())
        for tabName in df.keys():
            df[tabName].columns=list(map(cleanColumnName,df[tabName].columns))
            df[tabName]['fileName']=spreadsheet.split(".")[0]
            df[tabName]['tabName']=tabName
            if str(year) in tabName:
                print(tabName)
                #print(spreadsheet.split(".")[0], df[tabName].columns)
        df['budgetAll']=allBudgetData
        #allBudgetData=pd.concat(df.values(), sort=True)
        allBudgetData=pd.concat(df.values())
        allBudgetData['Year']=str(year)
        allBudgetData=allBudgetData.loc[allBudgetData['tabName'].str.contains(str(year))]
    return(allBudgetData)

def pullABunchOfYears():
    df=pd.DataFrame()
    listOfYears=[i for i in range(2013, 2021)]
    for year in listOfYears:
        budgetFiles=[i for i in os.listdir() if i.lower().startswith(str(year))]
        yearData=aggData(budgetFiles, year)
        df=pd.concat([df, yearData])
    return(df)
#allData=aggData(budgetFiles)
#print(allData.count().head())
df=pullABunchOfYears()


FY 2013
FY 2013 M1
FY 2013 Base
FY 2013 OCO
FY 2013 P1
FY 2013 Base
FY 2013 OCO
FY 2013 R1
FY 2013 Base
FY 2013 OCO
FY 2013 RF-1
FY 2013 Base RF-1
FY 2013 OCO RF-1
FY 2013 O1
FY 2013 Base O1
FY 2013 OCO O1
FY 2014
FY 2014 Base
FY 2014 Base
FY 2014 Base
FY 2014 Base
FY 2014 Base
FY 2015
FY 2015 Base
FY 2015 Base
FY 2015 Base
FY 2015 Base
FY 2015 Base
FY 2016
FY 2016 Base
FY 2016 OCO
FY 2016 Total
FY 2016 Base
FY 2016 OCO
FY 2016 Total
FY 2016 Base
FY 2016 OCO
FY 2016 Total
FY 2016 Base
FY 2016 OCO
FY 2016 Total
FY 2016 Base
FY 2016 OCO
FY 2016 Total
FY 2017
FY 2017 OCO
FY 2017 Base
FY 2017 OCO
FY 2017 Total
FY 2017 Base
FY 2017 OCO
FY 2017 Total
FY 2017 Base
FY 2017 OCO
FY 2017 Total
FY 2017 Base
FY 2017 OCO
FY 2017 Total
FY 2017 Base
FY 2017 OCO
FY 2017 Total
FY 2018
FY 2018 OCO
FY 2018 Base
FY 2018 OCO
FY 2018 Total
FY 2018 Base
FY 2018 OCO
FY 2018 Total
FY 2018 Base
FY 2018 OCO
FY 2018 Total
FY 2018 Base
FY 2018 OCO
FY 2018 Total
FY 2018 Base
FY 2018 OCO
FY 2018 Total
FY 2019
 FY 201

### Get 2017 Tabs and Values

In [22]:
def getYearData(df):
    data2017=df.loc[df['tabName'].str.contains("2017")]
    keepColumns=[i for i in data2017.columns if "2018" not in i and "2019" not in i]
    data2017=data2017[keepColumns].dropna(axis='columns',how='all')
    return(data2017)



### Standardize Budget Values

In [74]:
def genBudgetVars(year):
    dictOfCostVar={f"{year}c1": 'TOA Amount', f"{year}m1": f'FY {str(year)} (Base + OCO)', 
                   f"{year}o1": f'FY {year} Total', f"{year}p1": f"FY {str(year)} Total Amount",
                   f"{year}p1r": f"FY {year} (Base + OCO) Amount", f"{year}r1": f"FY {year} (Base + OCO)", f"{year}rf1": f"FY {year} (Base + OCO)"}
    return(dictOfCostVar)

def cleanBudgetVar(df, year):
    df[f'{str(year)} Budget Amount']=0
    for tab in df['tabName'].unique():
        #print(key)
        #print(len(df.loc[df['fileName']==key]))
        #print(df.loc[df['fileName']==key][dictOfCostVar[key]].sum())
        tabData=df.loc[df['tabName']==tab]
        sumColumns=[i for i in tabData.columns if str(year) in i]
        #print(sumColumns)
        if str(year)+"c1" in tabData['fileName'].unique():
            sumColumns=["TOA Amount"]
        #print(df[sumColumns].dtypes)
        df[f'{str(year)} Budget Amount']=df[sumColumns].apply(pd.to_numeric).max(axis=1)
        print(df[sumColumns].max())
        df['Top Spending']=df[f'{str(year)} Budget Amount']
        #df.loc[df['fileName']==key,f'{str(year)} Budget Amount']=df.loc[df['fileName']==key][f'{str(year)} Budget Amount']+df.loc[df['fileName']==key][dictOfCostVar[key]]
    return(df)

def cleanMultipleYears(df):
    cleanedDF=pd.DataFrame()
    for year in range(2015, 2019):
        year=str(year)
        yearSubset=df.loc[df['Year']==year]
        dictOfCostVar=genBudgetVars(year)
        addData=cleanBudgetVar(df, year)
        cleanedDF=pd.concat([cleanedDF, addData])
    #dropColumns=[i for i in cleanedDF.columns if ("OCO" in i) or ("Total" in i)]
    #cleanedDF=cleanedDF.drop(dropColumns)
    dropTabs=[i for i in cleanedDF['tabName'].unique() if ("OCO" in i) or ("Total" in i)]
    cleanedDF=cleanedDF.loc[~cleanedDF['tabName'].isin(dropTabs)]
    return(cleanedDF)
#dictOfCostVar=genBudgetVars()
#df=cleanBudgetVar(dictOfCostVar, data2017)
newDF=cleanMultipleYears(df)


FY 2015 (Base & OCO)                     NaN
FY 2015 (Base & OCO) Amount              NaN
FY 2015 (Base & OCO) Quantity            NaN
FY 2015 * OCO Enacted                    NaN
FY 2015 Base                      15412599.0
FY 2015 Base Amount               13990196.0
FY 2015 Base Enacted                     NaN
FY 2015 Base Enacted Amount              NaN
FY 2015 Base Enacted Quantity            NaN
FY 2015 Base Quantity                38234.0
FY 2015 Base*                            NaN
FY 2015 OCO Enacted                      NaN
FY 2015 OCO Enacted Amount *             NaN
FY 2015 OCO Enacted Quantity             NaN
FY 2015 Total Enacted                    NaN
FY 2015 Total Enacted Amount             NaN
FY 2015 Total Enacted Quantity           NaN
2015 Budget Amount                15412599.0
dtype: float64
FY 2015 (Base & OCO)                     NaN
FY 2015 (Base & OCO) Amount              NaN
FY 2015 (Base & OCO) Quantity            NaN
FY 2015 * OCO Enacted                   

FY 2015 (Base & OCO)                     NaN
FY 2015 (Base & OCO) Amount              NaN
FY 2015 (Base & OCO) Quantity            NaN
FY 2015 * OCO Enacted                    NaN
FY 2015 Base                      15412599.0
FY 2015 Base Amount               13990196.0
FY 2015 Base Enacted                     NaN
FY 2015 Base Enacted Amount              NaN
FY 2015 Base Enacted Quantity            NaN
FY 2015 Base Quantity                38234.0
FY 2015 Base*                            NaN
FY 2015 OCO Enacted                      NaN
FY 2015 OCO Enacted Amount *             NaN
FY 2015 OCO Enacted Quantity             NaN
FY 2015 Total Enacted                    NaN
FY 2015 Total Enacted Amount             NaN
FY 2015 Total Enacted Quantity           NaN
2015 Budget Amount                15412599.0
dtype: float64
FY 2015 (Base & OCO)                     NaN
FY 2015 (Base & OCO) Amount              NaN
FY 2015 (Base & OCO) Quantity            NaN
FY 2015 * OCO Enacted                   

FY 2015 (Base & OCO)                     NaN
FY 2015 (Base & OCO) Amount              NaN
FY 2015 (Base & OCO) Quantity            NaN
FY 2015 * OCO Enacted                    NaN
FY 2015 Base                      15412599.0
FY 2015 Base Amount               13990196.0
FY 2015 Base Enacted                     NaN
FY 2015 Base Enacted Amount              NaN
FY 2015 Base Enacted Quantity            NaN
FY 2015 Base Quantity                38234.0
FY 2015 Base*                            NaN
FY 2015 OCO Enacted                      NaN
FY 2015 OCO Enacted Amount *             NaN
FY 2015 OCO Enacted Quantity             NaN
FY 2015 Total Enacted                    NaN
FY 2015 Total Enacted Amount             NaN
FY 2015 Total Enacted Quantity           NaN
2015 Budget Amount                15412599.0
dtype: float64
FY 2015 (Base & OCO)                     NaN
FY 2015 (Base & OCO) Amount              NaN
FY 2015 (Base & OCO) Quantity            NaN
FY 2015 * OCO Enacted                   

FY 2015 (Base & OCO)                     NaN
FY 2015 (Base & OCO) Amount              NaN
FY 2015 (Base & OCO) Quantity            NaN
FY 2015 * OCO Enacted                    NaN
FY 2015 Base                      15412599.0
FY 2015 Base Amount               13990196.0
FY 2015 Base Enacted                     NaN
FY 2015 Base Enacted Amount              NaN
FY 2015 Base Enacted Quantity            NaN
FY 2015 Base Quantity                38234.0
FY 2015 Base*                            NaN
FY 2015 OCO Enacted                      NaN
FY 2015 OCO Enacted Amount *             NaN
FY 2015 OCO Enacted Quantity             NaN
FY 2015 Total Enacted                    NaN
FY 2015 Total Enacted Amount             NaN
FY 2015 Total Enacted Quantity           NaN
2015 Budget Amount                15412599.0
dtype: float64
FY 2015 (Base & OCO)                     NaN
FY 2015 (Base & OCO) Amount              NaN
FY 2015 (Base & OCO) Quantity            NaN
FY 2015 * OCO Enacted                   

FY 2016 Base                      14892683.0
FY 2016 Base + OCO                       NaN
FY 2016 Base + OCO Amount                NaN
FY 2016 Base + OCO Quantity              NaN
FY 2016 Base Amount               15038333.0
FY 2016 Base Enacted                     NaN
FY 2016 Base Enacted Amount              NaN
FY 2016 Base Enacted Quantity            NaN
FY 2016 Base Quantity                 6341.0
FY 2016 OCO                        4500666.0
FY 2016 OCO Amount                 3732499.0
FY 2016 OCO Enacted                      NaN
FY 2016 OCO Enacted Amount               NaN
FY 2016 OCO Enacted Quantity             NaN
FY 2016 OCO Quantity                746066.0
FY 2016 Total                     13129004.0
FY 2016 Total *                   15806502.0
FY 2016 Total Amount              18770832.0
FY 2016 Total Enacted                    NaN
FY 2016 Total Enacted Amount             NaN
FY 2016 Total Enacted Quantity           NaN
FY 2016 Total Quantity              746066.0
2016 Budge

...

FY 2018 (Base + OCO)                                                               NaN
FY 2018 (Base + OCO) Amount                                                        NaN
FY 2018 (Base + OCO) Quantity                                                      NaN
FY 2018 Base                                                                15317732.0
FY 2018 Base Amount                                                         16601513.0
FY 2018 Base Quantity                                                          10330.0
FY 2018 Emergency Requests** Emergency                                             NaN
FY 2018 Emergency Requests** Emergency Amount                                      NaN
FY 2018 Emergency Requests** Emergency Quantity                                    NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MDDE + Ship Repairs                       NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MDDE + Ship Repairs Amount                NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MD

FY 2018 (Base + OCO)                                                               NaN
FY 2018 (Base + OCO) Amount                                                        NaN
FY 2018 (Base + OCO) Quantity                                                      NaN
FY 2018 Base                                                                15317732.0
FY 2018 Base Amount                                                         16601513.0
FY 2018 Base Quantity                                                          10330.0
FY 2018 Emergency Requests** Emergency                                             NaN
FY 2018 Emergency Requests** Emergency Amount                                      NaN
FY 2018 Emergency Requests** Emergency Quantity                                    NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MDDE + Ship Repairs                       NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MDDE + Ship Repairs Amount                NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MD

FY 2018 (Base + OCO)                                                               NaN
FY 2018 (Base + OCO) Amount                                                        NaN
FY 2018 (Base + OCO) Quantity                                                      NaN
FY 2018 Base                                                                15317732.0
FY 2018 Base Amount                                                         16601513.0
FY 2018 Base Quantity                                                          10330.0
FY 2018 Emergency Requests** Emergency                                             NaN
FY 2018 Emergency Requests** Emergency Amount                                      NaN
FY 2018 Emergency Requests** Emergency Quantity                                    NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MDDE + Ship Repairs                       NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MDDE + Ship Repairs Amount                NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MD

FY 2018 (Base + OCO)                                                               NaN
FY 2018 (Base + OCO) Amount                                                        NaN
FY 2018 (Base + OCO) Quantity                                                      NaN
FY 2018 Base                                                                15317732.0
FY 2018 Base Amount                                                         16601513.0
FY 2018 Base Quantity                                                          10330.0
FY 2018 Emergency Requests** Emergency                                             NaN
FY 2018 Emergency Requests** Emergency Amount                                      NaN
FY 2018 Emergency Requests** Emergency Quantity                                    NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MDDE + Ship Repairs                       NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MDDE + Ship Repairs Amount                NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MD

FY 2018 (Base + OCO)                                                               NaN
FY 2018 (Base + OCO) Amount                                                        NaN
FY 2018 (Base + OCO) Quantity                                                      NaN
FY 2018 Base                                                                15317732.0
FY 2018 Base Amount                                                         16601513.0
FY 2018 Base Quantity                                                          10330.0
FY 2018 Emergency Requests** Emergency                                             NaN
FY 2018 Emergency Requests** Emergency Amount                                      NaN
FY 2018 Emergency Requests** Emergency Quantity                                    NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MDDE + Ship Repairs                       NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MDDE + Ship Repairs Amount                NaN
FY 2018 Less Enacted DIV B P.L.115-96*** MD

In [75]:
newDF.loc[(newDF['Organization']=="DODEA")].sum()

Appn Amount                                               14389900.0
Auth Amount                                               12345152.0
Auth For Appn Amount                                       2745400.0
Auth for Appn Amount                                      11644500.0
BSA                                                            936.0
Budget Activity                                                556.0
Emergency Disaster Relief Act of 2013                            0.0
Emergency Disaster Relief Act of 2013 Amount                     0.0
Emergency Disaster Relief Act of 2013 Quantity                   0.0
FY 2011 Actuals                                           12236580.0
FY 2011 Actuals Amount                                        5772.0
FY 2011 Actuals Quantity                                         0.0
FY 2012 (Base & OCO)                                             0.0
FY 2012 (Base & OCO) Amount                                      0.0
FY 2012 (Base & OCO) Quantity     

In [77]:
groupedBy=newDF.groupby(['Year', 'Organization', 'fileName', 'tabName']).sum()
print("did")
yearColumns=[i for i in groupedBy.columns if "20" in i]+["Top Spending"]
groupedBy=groupedBy[yearColumns].reset_index()
groupedBy.to_excel("aggregatedData.xlsx")
yearColumns=[i for i in groupedBy.columns if "2018" in i]
groupedBy.loc[(groupedBy['Organization']=="DODEA") & (groupedBy['tabName']=="2018c1")]['fileName']

did


Series([], Name: fileName, dtype: object)

### Accuracy Check

In [32]:
"${0:,.0f}".format(df['2017 Budget Amount'].sum())

'$618,157,868'

In [46]:
df['Organization'].value_counts()

NAVY     877
ARMY     737
AF       656
OSD       87
SOCOM     63
MDA       50
DHA       43
DLA       32
DISA      28
DARPA     17
DEFW      17
NSA       15
DTRA      11
TJS       10
CBDP      10
WHS        8
DODEA      8
DSCA       6
DIA        5
DSS        4
DHRA       4
IG         3
DCMA       3
OTE        3
NGA        2
DCAA       2
DTIC       2
DMACT      2
DECA       1
CMP        1
DPAA       1
DTSA       1
OEA        1
DFAS       1
DLSA       1
CAAF       1
DAU        1
Name: Organization, dtype: int64

In [33]:
def summarizeOrgInfo(df, org):
    print(f'Organization: {org}')
    budgetLines=(df.loc[df['Organization']==org])
    print(f'This has {len(budgetLines)} lines in the 2017 budget.')
    if len(budgetLines)>0:
        spendCats=budgetLines['fileName'].unique()
        print(f'They are from the following categories: {spendCats}.')
        print(f"Spending is as follows:")
        print(f"{budgetLines[['fileName','2017 Budget Amount']].groupby(['fileName']).sum()}.")
        print(f"Total budget is {budgetLines['2017 Budget Amount'].sum()}.")
              
              
    print("")

Organizations=list(set(df['Organization']))
for org in Organizations:
    summarizeOrgInfo(df, org)
    
    


Organization: nan
This has 0 lines in the 2017 budget.

Organization: OEA
This has 1 lines in the 2017 budget.
They are from the following categories: ['o1'].
Spending is as follows:
          2017 Budget Amount
fileName                    
o1                  311785.0.
Total budget is 311785.0.

Organization: OSD
This has 87 lines in the 2017 budget.
They are from the following categories: ['o1' 'r1' 'p1' 'c1'].
Spending is as follows:
          2017 Budget Amount
fileName                    
c1                      18.0
o1                 2122282.0
p1                  138289.0
r1                 4084372.0.
Total budget is 6344961.0.

Organization: DLSA
This has 1 lines in the 2017 budget.
They are from the following categories: ['o1'].
Spending is as follows:
          2017 Budget Amount
fileName                    
o1                  135896.0.
Total budget is 135896.0.

Organization: DEFW
This has 17 lines in the 2017 budget.
They are from the following categories: ['o1' 'p1' 'c1']

In [5]:
df.columns

Index(['AG / BSA', 'AG / Budget SubActivity (BSA) Title', 'AG Title',
       'Account', 'Account Short Title', 'Account Title', 'Add/ Non-Add',
       'Appn Amount', 'Auth Amount', 'Auth For Appn Amount',
       ...
       'SAG Title', 'State Country', 'State Country Title', 'TOA Amount',
       'TOA Base Request with CR Adj *',
       'TOA Total Base Request with CR Adj *', 'Treasury Agency', 'Year',
       'fileName', 'tabName'],
      dtype='object', length=238)

In [9]:
df.loc[df['Organization']=="DISA"].to_excel("DISA.xlsx")